<a href="https://colab.research.google.com/github/arol9204/English_Speaking_AI_Examiner/blob/main/English_Speaking_AI_examiner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain and LLM dolly

In [1]:
!pip install --upgrade langchain

                                              0.0/1.7 MB ? eta -:--:--
                                              0.0/1.7 MB ? eta -:--:--
     --                                       0.1/1.7 MB 1.3 MB/s eta 0:00:02
     ------------                             0.5/1.7 MB 4.1 MB/s eta 0:00:01
     ----------------------                   1.0/1.7 MB 6.2 MB/s eta 0:00:01
     ----------------------------------       1.5/1.7 MB 7.3 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 7.3 MB/s eta 0:00:00
                                              0.0/2.0 MB ? eta -:--:--
     ---------                                0.5/2.0 MB 10.5 MB/s eta 0:00:01
     ---------------------                    1.1/2.0 MB 14.2 MB/s eta 0:00:01
     ---------------------------------        1.7/2.0 MB 12.0 MB/s eta 0:00:01
     ---------------------------------------- 2.0/2.0 MB 11.9 MB/s eta 0:00:00
                                              0.0/374.5 kB ? eta -:--:--
     ---


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install "transformers[torch]"

^C
Note: you may need to restart the kernel to use updated packages.


                                              0.0/7.6 MB ? eta -:--:--
                                              0.0/7.6 MB ? eta -:--:--
                                              0.1/7.6 MB 762.6 kB/s eta 0:00:10
     --                                       0.5/7.6 MB 3.1 MB/s eta 0:00:03
     ----                                     0.9/7.6 MB 4.6 MB/s eta 0:00:02
     -------                                  1.4/7.6 MB 5.9 MB/s eta 0:00:02
     ----------                               2.0/7.6 MB 7.5 MB/s eta 0:00:01
     -------------                            2.6/7.6 MB 8.0 MB/s eta 0:00:01
     ----------------                         3.2/7.6 MB 8.5 MB/s eta 0:00:01
     -------------------                      3.8/7.6 MB 9.0 MB/s eta 0:00:01
     ----------------------                   4.4/7.6 MB 9.0 MB/s eta 0:00:01
     -------------------------                4.8/7.6 MB 9.3 MB/s eta 0:00:01
     ----------------------------             5.4/7.6 MB 9.8 MB/s eta 0:00:0


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
from transformers import pipeline


# databricks/dolly-v2-3b
# databricks/dolly-v2-7b
# databricks/dolly-v2-2-8b
# databricks/dolly-v2-12b

# OpenAssistant/falcon-7b-sft-mix-2000
# OpenAssistant/falcon-40b-sft-mix-1226


generate_text = pipeline(model="databricks/dolly-v2-3b",
                         torch_dtype=torch.bfloat16,
                         trust_remote_code=True,
                         device_map="auto",
                         return_full_text=True)

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# Installing Whisper for speaking-to-text trasncript

In [ ]:
# Install the latest commit from Github whisper repository
! pip install git+https://github.com/openai/whisper.git -q

In [ ]:
# Installing command-line tool ffmpeg
!sudo apt update && sudo apt install ffmpeg

# Install the Web UI Toolkit

In [ ]:
! pip install gradio -q

# The App

In [ ]:
import whisper
import gradio as gr
import time

In [ ]:
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

English_review_template = """You are a professional examiner who assesses the English language proficiency of candidates taking an English test.
You asked this question: {question} \
The answer was: {text}.

Evaluate the answer based on the following criteria:\

- Coherence: Assess how well the candidate's answer is connected to the question.
- Word Choice: Assess if the candidate uses appropriate words and phrases to communicate.
- Vocabulary Range: Assess if the candidate uses a variety of vocabulary.
- Sentence Structure: Assess the candidate's ability to use a variety of sentence structures appropriately.
- Grammatical Accuracy: Explain how well did the candidate formed grammatically correct sentences?
- Grammatical Errors: List and explain each grammar error in the candidate's answer.
"""

prompt_template = PromptTemplate.from_template(English_review_template)

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt_template)

In [ ]:
def English_feedback(Question, audio):
  model = whisper.load_model("base")
  text = model.transcribe(audio)
  response = llm_chain.run({'text': text, 'question': Question})
  return text["text"], response

In [ ]:
demo = gr.Interface(
    title = 'English Speaking AI-examiner ',
    fn=English_feedback,
    inputs=["text", gr.inputs.Audio(source="microphone", type="filepath")],
    outputs=[ "textbox", "textbox"])

demo.launch(share=True)